In [1]:
import sys
from pathlib import Path
sys.path.append(r"C:\Users\kotoub.a\AppData\Roaming\Python\Python312\site-packages")
import duckdb, os


In [ ]:

# DB dans le même dossier que ce notebook ? sinon ajuste ce chemin absolu
BASE_DIR = Path.cwd()  # <- par défaut: dossier du notebook
DB_PATH  = BASE_DIR / "warehouse.duckdb"

# Si ton ETL a écrit le DW ailleurs, **mets le chemin absolu** ci-dessous :
# DB_PATH = Path(r"D:\MSPR_CODE\MSPR_BLOC3_MASTER1\etl\warehouse.duckdb")

print("DuckDB utilisé :", DB_PATH.resolve())

con = duckdb.connect(str(DB_PATH))
con.execute("PRAGMA database_list").df()  # vérif du fichier réellement ouvert


In [ ]:
# Lister les tables
con.execute("SHOW TABLES").df()

In [ ]:
# Décrire une table (ex: fact_election_tour1)
con.execute("DESCRIBE fact_election_tour1").df()

In [ ]:
# ⚠️ Adapte les noms de colonnes si besoin (regarde la cellule "DESCRIBE")
QUERY = """
WITH base AS (
    SELECT
        t1.code_departement,
        t1.libelle_departement,
        t1.inscrits,
        t1.votants,
        t1.blancs,
        t1.nuls,
        t1.exprimes,
        c.taux_chomage
    FROM fact_election_tour1 AS t1
    LEFT JOIN fact_chomage AS c
        ON c.code_departement = t1.code_departement
),
feat AS (
    SELECT
        *,
        CAST(votants AS DOUBLE)/NULLIF(inscrits,0)    AS taux_participation,
        CAST(blancs  AS DOUBLE)/NULLIF(votants ,0)    AS taux_blancs_sur_votants,
        CAST(nuls    AS DOUBLE)/NULLIF(votants ,0)    AS taux_nuls_sur_votants,
        CAST(exprimes AS DOUBLE)/NULLIF(votants ,0)   AS taux_exprimes_sur_votants
    FROM base
)
SELECT *
FROM feat
WHERE inscrits IS NOT NULL AND votants IS NOT NULL AND exprimes IS NOT NULL
"""

df = con.execute(QUERY).df()
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# - Si tu crées une cible binaire: participation > 60% ?
df["target"] = (df["taux_participation"] >= 0.60).astype(int)

# Features numériques candidates (adapte selon ton besoin)
feature_cols = [
    "inscrits","votants","blancs","nuls","exprimes",
    "taux_chomage","taux_participation","taux_blancs_sur_votants",
    "taux_nuls_sur_votants","taux_exprimes_sur_votants"
]

X = df[feature_cols].copy()
y = df["target"].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("clf", LogisticRegression(max_iter=200, n_jobs=None))
])
pipe

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

pipe.fit(X_train, y_train)

y_pred  = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

In [ ]:
from joblib import dump
MODEL_PATH = BASE_DIR / "model_participation.joblib"
dump(pipe, MODEL_PATH)
print("Modèle sauvegardé :", MODEL_PATH)

In [ ]:
from joblib import load
pipe2 = load(MODEL_PATH)

# Prédiction sur 5 lignes pour vérifier que tout marche
pipe2.predict(X_test.iloc[:5]), pipe2.predict_proba(X_test.iloc[:5])[:,1]